## Домашнее задание к Уроку 3

Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру.
Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
    
    Наименование вакансии.
    Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.
    По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение)

Структура должна быть одинаковая для вакансий с всех сайтов. Общий результат можно вывести с помощью dataFrame
через pandas, сохранить в json, либо csv.

Домашнее задание нужно приложить как ссылка на репозиторий. Для этого вставьте ссылку на репозиторий в текстовое окно и нажмите кнопку "сдать".

Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий имеет readme.md с описанием и структурой проекта
- есть скриншоты результата сбора данных и обращения к БД

In [2]:
# Импортируем необходимые библиотеки
import requests
import pprint
from time import sleep
from copy import deepcopy
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from datetime import date, datetime
import pymongo

In [3]:
# Задаим типовую структуру вакансии для наполнения списка вакансий
tmp_vacancy_dict = {
    # Наименование вакансии
    'vacancy': '',
    # Зарплата включая минимум, максимум и валюту
    'salary': {'min_salary': 0., 'max_salary': 0., 'currency': ''},
    # Ссылка на описание вакансии
    'vacancy_link': '',
    # источник вакансии
    'vacancy_source': 'hh.ru',
    # Наименование работодателя
    'name_of_employer': '',
    # Место работы (город или удаленка)
    'city_of_work': '',
    # условия работы - полный день, удаленка
    'type_of_work': ''
}

In [4]:
# Зададим заголовок для выполнения запроса к сайтам
user_agent = UserAgent()
headers = {'User-Agent': user_agent.chrome}

In [5]:
# Зададим словарь url сайтов поиска работы
url_of_headhunter_companies = {
    'HeadHunter': 'https://api.hh.ru/vacancies',
    'Superjob': 'https://www.superjob.ru/vacancy/search/',
    'Работа.ру': 'https://www.rabota.ru/vacancy/'
}

In [6]:
# Зададим словарь параметров для запросов на сайте HeadHunter
headhunter_request_params = {
    'text': 'Python',
    'area': '1',
    'items_on_page': '20',
    'page': '0'
}

In [7]:
# Зададим словарь параметров для запросов на сайте Superjob
superjob_request_params = {
    'keywords': 'Python',
    'geo%5Bt%5D%5B0%5D': '4'
}

In [8]:
# Зададим словарь параметров для запросов на сайте Работа.ру
rabotaru_request_params = {
    'query': 'Python',
    'sort': 'relevance'
}

#### Подключимся и создадим структуру БД в MangoDB

In [9]:
# Подключимся к локальной MongoDB 
local_mdb_client = pymongo.MongoClient('mongodb://localhost:27017/')
# Создадим/Подключимся к БД vacancy_db
vacancy_db = local_mdb_client['vacancy_db']
# Создадим/Подключим коллекции по именам источников вакансий
headhunter_collectiion = vacancy_db['headhunter']
superjob_collection = vacancy_db['superjob']
rabotaru_collection = vacancy_db['rabotaru']

#### Создадим необходимые функции

In [10]:
# Функция проверки наличия в строке отдельных чисел
def is_digit_in_string(source_string):
    word_list = source_string.split()
    
    for word in word_list:
        if word.isdigit():
            return True
        
    return False

In [11]:
# Функция выделения размера зарплаты из строки
def get_digit_from_string(source_string):
    str_to_list = source_string.split()
    idx_numeric_list = []
    number_sring = ''
    
    for i in range(len(str_to_list)):
        if str_to_list[i].isdigit():
            idx_numeric_list.append(i)
    
    for idx in idx_numeric_list:
        number_sring += str_to_list[idx]
    
    return float(number_sring)

In [12]:
# Функция получения типа валюты из строки
def get_currency(currency_string):
    
    if currency_string.lower().find('руб') >= 0:
        return 'Руб.'
    elif currency_string.find('₽') >= 0:
        return 'Руб.'
    elif currency_string.find('$') >= 0:
        return 'USD'
    
    return ''

In [13]:
# Функция бработки строки о зарплате
def get_salary_dict(salary_string):
    
    if salary_string.lower().startswith('по'):
        return {'min_salary': 0.0, 
                'max_salary': 0.0, 
                'currency': ''}.copy()
    elif salary_string.lower().startswith('от'):
        return {'min_salary': get_digit_from_string(salary_string), 
                'max_salary': 0.0, 
                'currency': get_currency(salary_string)}.copy()
    elif salary_string.lower().startswith('до'):
        return {'min_salary': 0.0, 
                'max_salary': get_digit_from_string(salary_string), 
                'currency': get_currency(salary_string)}
    else:
        if salary_string.find('—') != -1:
            two_numbers_string_list = salary_string.split('—')
            return {'min_salary': get_digit_from_string(two_numbers_string_list[0]), 
                    'max_salary': get_digit_from_string(two_numbers_string_list[1]), 
                    'currency': get_currency(salary_string)}
        else:
            return {'min_salary': 0.0, 
                    'max_salary': get_digit_from_string(salary_string), 
                    'currency': get_currency(salary_string)}
    
    return {}

### Обработка вакансий разработчика python для города Москвы с сайта HeadHunter

#### hh защищаеться от стандартного парсинга и выдает ошибку 404 или 403 что видно ниже

In [14]:
# Выполним запрос к сайту
headhunter_url = 'https://hh.ru/search/vacancy?text=Python&area=1'
headhunter_respond = requests.post(url=headhunter_url, headers=headers)

In [15]:
headhunter_respond

<Response [403]>

#### Парсинг hh будет реализован через стандартный hh api 

In [16]:
%%time
# С данного сайта получить данные вакансий можно через api в json формате
# Задаим url запроса из словаря
headhunter_url = url_of_headhunter_companies['HeadHunter']

# Инициализируем список словарей для хранения вакансий
headhunter_vacancy_list = []

# Организуем цикл сбора информации
for i in range(40):
    # Чтобы избежвть ошики о превышения коичество запросов
    sleep(2)
    
    # Занаосим в параметры запроса номер страницы
    headhunter_request_params['page'] = str(i)
    
    # Выполним запрос к сайту, ответ преобразуем в json а затем в список
    headhunter_respond = requests.get(url=headhunter_url, params=headhunter_request_params)
    headhunter_respond_json_list = (headhunter_respond.json())['items']
    
    if headhunter_respond.ok:
        # Для каждой вакансии
        for one_vacancy in headhunter_respond_json_list:
            # Заполняем словарь вакансии
            tmp_vacancy_dict['vacancy'] = one_vacancy['name']
        
            if one_vacancy['salary'] is None:
                tmp_vacancy_dict['salary']['min_salary'] = 0.0
                tmp_vacancy_dict['salary']['max_salary'] = 0.0
                tmp_vacancy_dict['salary']['currency'] = ''
            else:
                if one_vacancy['salary']['from'] is None:
                    tmp_vacancy_dict['salary']['min_salary'] = 0.0
                else:
                    tmp_vacancy_dict['salary']['min_salary'] = float(one_vacancy['salary']['from'])
            
                if one_vacancy['salary']['to'] is None:
                    tmp_vacancy_dict['salary']['max_salary'] = 0.0
                else:
                    tmp_vacancy_dict['salary']['max_salary'] = float(one_vacancy['salary']['to'])
            
                tmp_vacancy_dict['salary']['currency'] = one_vacancy['salary']['currency']
            
            tmp_vacancy_dict['vacancy_link'] = one_vacancy['alternate_url']
            tmp_vacancy_dict['name_of_employer'] = one_vacancy['employer']['name']
            tmp_vacancy_dict['city_of_work'] = one_vacancy['area']['name']
            tmp_vacancy_dict['type_of_work'] = one_vacancy['schedule']['name']
        
            # Добавляем вакансю в список вакансий
            headhunter_vacancy_list.append(deepcopy(tmp_vacancy_dict))
    else:
        break

Wall time: 1min 35s


In [17]:
# Выведем количество полученных вакансий
len(headhunter_vacancy_list)

800

In [28]:
headhunter_vacancy_list

[{'vacancy': 'Python-разработчик (Django, DRF)',
  'salary': {'min_salary': 60000.0, 'max_salary': 220000.0, 'currency': 'RUR'},
  'vacancy_link': 'https://hh.ru/vacancy/70705466',
  'vacancy_source': 'hh.ru',
  'name_of_employer': 'Фабрика Решений',
  'city_of_work': 'Москва',
  'type_of_work': 'Удаленная работа',
  '_id': ObjectId('63f8cfee2676ceaada4acb88')},
 {'vacancy': 'Разработчик Python',
  'salary': {'min_salary': 200000.0,
   'max_salary': 250000.0,
   'currency': 'RUR'},
  'vacancy_link': 'https://hh.ru/vacancy/77091465',
  'vacancy_source': 'hh.ru',
  'name_of_employer': 'Нетопия',
  'city_of_work': 'Москва',
  'type_of_work': 'Полный день',
  '_id': ObjectId('63f8cfee2676ceaada4acb89')},
 {'vacancy': 'Разработчик C# / Python',
  'salary': {'min_salary': 300000.0,
   'max_salary': 500000.0,
   'currency': 'RUR'},
  'vacancy_link': 'https://hh.ru/vacancy/71147348',
  'vacancy_source': 'hh.ru',
  'name_of_employer': 'Дубайт',
  'city_of_work': 'Москва',
  'type_of_work': 'Пол

In [18]:
# Сохраним вакансии в БД
x = headhunter_collectiion.insert_many(headhunter_vacancy_list)

### Обработка вакансий разработчика python для города Москвы с сайта Superjon

In [19]:
# Задаим url запроса из словаря
superjob_url = url_of_headhunter_companies['Superjob']
# Выполним запрос к сайту, ответ преобразуем в json а затем в список
superjob_response = requests.get(url=superjob_url, params=superjob_request_params)

In [20]:
# Преобразуем ответ в формат HTML
superjob_dom = bs(superjob_response.text, 'html.parser')

In [21]:
superjob_dom

<!DOCTYPE html>
<html class="no-js" lang="ru"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><title>Поиск работы в Москве. 47 вакансий | Superjob</title><meta content="Поиск вакансий на портале superjob.ru в Москве. Удобный поиск работы, множество фильтров, вакансии на карте рядом с домом, большая база работодателей. Регистрация бесплатно!" name="description"/><meta content="работа, вакансии, резюме, поиск работы" name="keywords"/><meta content="notranslate" name="google"/><meta content="6fa953a8f9" name="verify-admitad"/><meta content="width=device-width" name="viewport"/><meta content="telephone=no" name="format-detection"/><meta content="browserconfig.xml" name="msapplication-config"/><link href="/fstc/desktop/favicon.ico" rel="shortcut icon"/><link href="/fstc/desktop/favicon.png" rel="icon" sizes="558x558" type="image/png"/><link href="/fstc/desktop/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/fstc/desktop/favicon-3

In [22]:
# Соберем в один список результаты полиска вакансий
tag_div_vacancy_list = superjob_dom.select('div._1bobf.f-test-vacancy-item')

In [23]:
# Выведем количество полученных элементов
len(tag_div_vacancy_list)

40

In [24]:
# Обработаем все полученные вакансии
superjob_vacancy_list = []

tmp_vacancy_dict['vacancy_source'] = 'superjob.ru'
tmp_vacancy_dict['type_of_work'] = ''

for tag_vacancy in tag_div_vacancy_list:
    # Теперь определим нужные блоки формы вакансии
    superjob_one_vacancy = tag_vacancy.select('span a')
    
    # Получим наименование вакансии
    tmp_vacancy_dict['vacancy'] = superjob_one_vacancy[0].text
    # Получим ссылку на вакансию
    tmp_vacancy_dict['vacancy_link'] = 'https://www.superjob.ru' + superjob_one_vacancy[0]['href']
    # Получим работодателя
    # Встречаються обьявления без указания работодателя в заголовке
    if len(superjob_one_vacancy) > 1 :
        tmp_vacancy_dict['name_of_employer'] = superjob_one_vacancy[1].text
    else:
        tmp_vacancy_dict['name_of_employer'] = 'Смотрите по ссылке на описание вакансии'
        
    # Получим город работы
    if (tag_vacancy.select('span div div'))[0].text == 'Удаленная работа':
        tmp_vacancy_dict['city_of_work'] = ''
        tmp_vacancy_dict['type_of_work'] = (tag_vacancy.select('span div div'))[0].text
    else:
        tmp_vacancy_dict['city_of_work'] = (tag_vacancy.select('span div div'))[0].text
        tmp_vacancy_dict['type_of_work'] = 'офис'

    # Занесем информацию по зарплате
    superjob_vacancy_salary = (tag_vacancy.select('div.f-test-text-company-item-salary span'))[0].text
    vacancy_salary = superjob_vacancy_salary.replace('\xa0', ' ')
    
    # Обработаем данные по зарплате
    salary_dict = get_salary_dict(vacancy_salary)
    if len(salary_dict) > 0:
        tmp_vacancy_dict['salary']['min_salary'] = salary_dict['min_salary']
        tmp_vacancy_dict['salary']['max_salary'] = salary_dict['max_salary']
        tmp_vacancy_dict['salary']['currency'] = salary_dict['currency']
    else:
        tmp_vacancy_dict['salary']['min_salary'] = 0.0
        tmp_vacancy_dict['salary']['max_salary'] = 0.0
        tmp_vacancy_dict['salary']['currency'] = ''

    superjob_vacancy_list.append(deepcopy(tmp_vacancy_dict))
    

In [25]:
# Выведем количество полученных вакансий
len(superjob_vacancy_list)

40

In [27]:
superjob_vacancy_list

[{'vacancy': 'Эксперт (SQL, Python) в отдел разработки систем принятия кредитного решения',
  'salary': {'min_salary': 0.0, 'max_salary': 0.0, 'currency': ''},
  'vacancy_link': 'https://www.superjob.ru/vakansii/ekspert-45802697.html',
  'vacancy_source': 'superjob.ru',
  'name_of_employer': 'Банк ВТБ',
  'city_of_work': 'Москва',
  'type_of_work': 'офис',
  '_id': ObjectId('63f8d0ec2676ceaada4acea8')},
 {'vacancy': 'Специалист службы поддержки с техническими знаниями (Браузер)',
  'salary': {'min_salary': 15000.0, 'max_salary': 0.0, 'currency': 'Руб.'},
  'vacancy_link': 'https://www.superjob.ru/vakansii/specialist-sluzhby-podderzhki-s-tehnicheskimi-znaniyami-39820624.html',
  'vacancy_source': 'superjob.ru',
  'name_of_employer': 'Яндекс',
  'city_of_work': 'Москва',
  'type_of_work': 'офис',
  '_id': ObjectId('63f8d0ec2676ceaada4acea9')},
 {'vacancy': 'Сервис-инженер',
  'salary': {'min_salary': 60000.0, 'max_salary': 0.0, 'currency': 'Руб.'},
  'vacancy_link': 'https://www.superjob

In [26]:
# Сохраним вакансии в БД
x = superjob_collection.insert_many(superjob_vacancy_list)

### Обработка вакансий разработчика python для города Москвы с сайта Работа.ру

In [29]:
# Задаим url запроса из словаря
rabotaru_url = url_of_headhunter_companies['Работа.ру']
# Выполним запрос к сайту, ответ преобразуем в json а затем в список
rabotaru_response = requests.get(url=rabotaru_url, params=rabotaru_request_params)

In [30]:
rabotaru_dom = bs(rabotaru_response.text, 'html.parser')

In [31]:
rabotaru_dom

<!DOCTYPE html>

<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D,%22prefix%22:%7B%22ssr%22:%22og:%20http://ogp.me/ns#%22%7D%7D" data-n-head-ssr="" lang="ru" prefix="og: http://ogp.me/ns#">
<head>
<title>Python в Москве - свежие вакансии Python на Rabota.ru</title><meta charset="utf-8" data-n-head="ssr"/><meta content="viewport-fit=cover, width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" data-n-head="ssr" name="viewport"/><meta content="#3495db" data-hid="theme-color" data-n-head="ssr" name="theme-color"/><meta content="IE=edge" data-n-head="ssr" http-equiv="X-UA-Compatible"/><meta content="ru_RU" data-hid="og:locale" data-n-head="ssr" name="og:locale" property="og:locale"/><meta content="website" data-hid="og:type" data-n-head="ssr" name="og:type" property="og:type"/><meta content="https://www.rabota.ru/static/images/artboard_ogImage.png" data-hid="og:image" data-n-head="ssr" property="og:image"/><meta content="968" data-hid="og:image:width" data-n-head="

In [32]:
# Получим все блоки описания вакансий
rabotaru_tag_vacancy_list = rabotaru_dom.select('div article.vacancy-preview-card')

In [33]:
# Выведем количество полученых вакансий
len(rabotaru_tag_vacancy_list)

20

In [34]:
# Обработаем вакансии
# Создадим пустой список для обработанных вакансий
rabotaru_vacancy_list = []
tmp_vacancy_dict['vacancy_source'] = 'rabota.ru'


for one_vacancy_tag in rabotaru_tag_vacancy_list:
    
    # Получим наименование вакансии
    tmp_vacancy_dict['vacancy'] = (one_vacancy_tag.select('a.vacancy-preview-card__title_border')[0].text).replace('\n', '').strip()
    # Получим ссылку на вакансию
    tmp_vacancy_dict['vacancy_link'] = 'https://www.rabota.ru'+ one_vacancy_tag.select('a.vacancy-preview-card__title_border')[0]['href']
    # Получим работодателя
    tmp_vacancy_dict['name_of_employer'] = (one_vacancy_tag.select('span.vacancy-preview-card__company-name')[0].text).replace('\n', '').strip()
    # Получим город работы и тип работы
    if one_vacancy_tag.select('span.vacancy-preview-location__address-text')[0].text == 'Удаленная работа':
        tmp_vacancy_dict['city_of_work'] = ''
        tmp_vacancy_dict['type_of_work'] = 'Удаленная работа'
    else:
        citi_of_work = (one_vacancy_tag.select('span.vacancy-preview-location__address-text')[0].text).replace('\n', '').strip()
        if citi_of_work.find('Москва') == -1:  
            tmp_vacancy_dict['city_of_work'] = citi_of_work + ', Москва'
        else:
            tmp_vacancy_dict['city_of_work'] = citi_of_work
            
        tmp_vacancy_dict['type_of_work'] = 'офис'

    # Обработаем данные по зарплате
    vacancy_salary = (one_vacancy_tag.select('div.vacancy-preview-card__salary a')[0].text).replace ('\xa0', ' ')
    salary_dict = get_salary_dict(vacancy_salary)
    if len(salary_dict) > 0:
        tmp_vacancy_dict['salary']['min_salary'] = salary_dict['min_salary']
        tmp_vacancy_dict['salary']['max_salary'] = salary_dict['max_salary']
        tmp_vacancy_dict['salary']['currency'] = salary_dict['currency']
    else:
        mp_vacancy_dict['salary']['min_salary'] = 0.0
        tmp_vacancy_dict['salary']['max_salary'] = 0.0
        tmp_vacancy_dict['salary']['currency'] = ''
    
    rabotaru_vacancy_list.append(deepcopy(tmp_vacancy_dict))
    


In [35]:
rabotaru_vacancy_list

[{'vacancy': 'DBA Engineer (PostgreSQL)',
  'salary': {'min_salary': 0.0, 'max_salary': 0.0, 'currency': ''},
  'vacancy_link': 'https://www.rabota.ru/vacancy/47298372/?search_id=1677250938948cabtyom8m2h',
  'vacancy_source': 'rabota.ru',
  'name_of_employer': 'Работа.ру',
  'city_of_work': 'м. Белорусская, Москва',
  'type_of_work': 'офис'},
 {'vacancy': 'Тестировщик программного обеспечения',
  'salary': {'min_salary': 0.0, 'max_salary': 0.0, 'currency': ''},
  'vacancy_link': 'https://www.rabota.ru/vacancy/47309429/?search_id=1677250938948cabtyom8m2h',
  'vacancy_source': 'rabota.ru',
  'name_of_employer': 'ООО "АНсистемс"',
  'city_of_work': 'м. Семёновская, Москва',
  'type_of_work': 'офис'},
 {'vacancy': 'Разработчик-стажёр моделей (Data Scientist)',
  'salary': {'min_salary': 25000.0, 'max_salary': 57000.0, 'currency': 'Руб.'},
  'vacancy_link': 'https://www.rabota.ru/vacancy/43332615/?search_id=1677250938948cabtyom8m2h',
  'vacancy_source': 'rabota.ru',
  'name_of_employer': 'К